### Librerias

In [1]:
import pandas as pd  
import os
import numpy as np
import datetime as dt # para gestión de fechas
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
plt.style.use('ggplot')
from matplotlib.pyplot import figure

### Importar módulos creados por nosotros

In [2]:
import funciones as fn #funciones es el nombre del script dnd hemos creado las funciones
#from funciones import * 
#from scripts.librerias import *

### Carga de datos

In [3]:
path_dataset = 'D:\\Desktop\\Reto06\\datosOriginales\\' # ruta donde se encuentran los datos
file = 'EDERJAKIN_LA_Datos_2021.xlsm'
fichero = pd.ExcelFile(path_dataset + file)
nombres_hojas = fichero.sheet_names

In [4]:
# leer todas las hojas
todo  = pd.read_excel(os.path.join(path_dataset,file), sheet_name = nombres_hojas)

# separar
for i in todo.keys():
    globals()['%s' % i.replace(" ", "_").lower()] = todo[i]

# Limpieza 

### CORRECCION DEL TIPO DE LAS VARIABLES

In [5]:
#usuarios:
usuarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           420 non-null    int64  
 1   useranonimo_name  420 non-null    object 
 2   active_flag       420 non-null    int64  
 3   language          420 non-null    object 
 4   last_access       0 non-null      float64
 5   mdl_user_id       405 non-null    float64
 6   role_admin        420 non-null    int64  
 7   role_manager      420 non-null    int64  
 8   role_student      420 non-null    int64  
 9   role_tutor        420 non-null    int64  
 10  planta_name       420 non-null    object 
dtypes: float64(2), int64(6), object(3)
memory usage: 36.2+ KB


In [6]:
fn.corregir_tipo(df = usuarios, columnas = [ 'language', 'planta_name'], tipo = 'category') #las pasamos a tipo category
fn.corregir_tipo(df = usuarios, columnas = ['active_flag',  'role_admin','role_manager','role_student','role_tutor'], tipo = 'bool') #a tipo bool
usuarios.info() #correcto

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   user_id           420 non-null    int64   
 1   useranonimo_name  420 non-null    object  
 2   active_flag       420 non-null    bool    
 3   language          420 non-null    category
 4   last_access       0 non-null      float64 
 5   mdl_user_id       405 non-null    float64 
 6   role_admin        420 non-null    bool    
 7   role_manager      420 non-null    bool    
 8   role_student      420 non-null    bool    
 9   role_tutor        420 non-null    bool    
 10  planta_name       420 non-null    category
dtypes: bool(5), category(2), float64(2), int64(1), object(1)
memory usage: 16.6+ KB


In [7]:
#quitamos columnas Unnamed:
fn.eliminar_unnamed(usuarios)
usuarios.head() #correcto

,user_id,useranonimo_name,active_flag,language,last_access,mdl_user_id,role_admin,role_manager,role_student,role_tutor,planta_name
0,3,username_03,True,es,NaN,13.0,False,False,True,False,ABP Bergara
1,4,username_04,True,es,NaN,14.0,False,False,True,False,Fagorederlan
2,5,username_05,True,es,NaN,15.0,False,False,True,False,ABP Arrasate
3,6,username_06,True,es,NaN,17.0,False,False,True,False,Fagorederlan
4,7,username_07,True,es,NaN,18.0,False,False,True,False,ABP Arrasate


In [8]:
#tutores_perfil
tutores_perfil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   perfil_id    188 non-null    int64 
 1   user_id      188 non-null    int64 
 2   perfil_name  118 non-null    object
dtypes: int64(2), object(1)
memory usage: 4.5+ KB


In [9]:
fn.corregir_tipo(df= tutores_perfil, columnas= ['perfil_id'], tipo= 'category') #las pasamos a tipo category
fn.eliminar_unnamed(tutores_perfil) #eliminamos columnas unnamed
tutores_perfil.info() #correcto

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   perfil_id    188 non-null    category
 1   user_id      188 non-null    int64   
 2   perfil_name  118 non-null    object  
dtypes: category(1), int64(1), object(1)
memory usage: 3.4+ KB


In [10]:
#Conocimientos
conocimientos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   conocimiento_id              155 non-null    int64 
 1   conocimiento_name            155 non-null    object
 2   color                        21 non-null     object
 3   created_by                   155 non-null    int64 
 4   description                  17 non-null     object
 5   categoria_conocimiento_id    155 non-null    int64 
 6   referencia                   33 non-null     object
 7   categoria_conocimiento_name  133 non-null    object
dtypes: int64(3), object(5)
memory usage: 9.8+ KB


In [11]:
fn.corregir_tipo(df= conocimientos, columnas= ['color', 'categoria_conocimiento_id','created_by'], tipo= 'category') #las pasamos a category
fn.eliminar_unnamed(conocimientos) #quitamos columnas 'unnamed'
conocimientos.info() #correcto

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   conocimiento_id              155 non-null    category
 1   conocimiento_name            155 non-null    object  
 2   color                        21 non-null     category
 3   created_by                   155 non-null    category
 4   description                  17 non-null     object  
 5   categoria_conocimiento_id    155 non-null    category
 6   referencia                   33 non-null     object  
 7   categoria_conocimiento_name  133 non-null    object  
dtypes: category(4), object(4)
memory usage: 14.2+ KB


In [12]:
#Cursos
cursos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   course_id         139 non-null    int64 
 1   course_name       139 non-null    object
 2   category          139 non-null    int64 
 3   hours             139 non-null    int64 
 4   course_shortname  139 non-null    object
 5   sortorder         139 non-null    int64 
 6   summary           44 non-null     object
dtypes: int64(4), object(3)
memory usage: 7.7+ KB


In [13]:
fn.corregir_tipo(df= cursos, columnas= ['category'], tipo= 'category') #las pasamos a tipo category
fn.eliminar_unnamed(cursos) #eliminamos columnas unnamed
cursos.info() #correcto

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   course_id         139 non-null    category
 1   course_name       139 non-null    object  
 2   category          139 non-null    category
 3   hours             139 non-null    int64   
 4   course_shortname  139 non-null    object  
 5   sortorder         139 non-null    int64   
 6   summary           44 non-null     object  
dtypes: category(2), int64(2), object(3)
memory usage: 12.8+ KB


In [14]:
#Programas
programas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   program_id           110 non-null    int64 
 1   program_name         110 non-null    object
 2   program_category_id  109 non-null    object
dtypes: int64(1), object(2)
memory usage: 2.7+ KB


In [15]:
fn.corregir_tipo(df= programas, columnas= ['program_id', 'program_category_id', ], tipo= 'category') #las pasamos a tipo category
fn.eliminar_unnamed(programas) #eliminamos columnas unnamed
programas.info() #correcto

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   program_id           110 non-null    category
 1   program_name         110 non-null    object  
 2   program_category_id  109 non-null    category
dtypes: category(2), object(1)
memory usage: 7.4+ KB


In [86]:
#Cursos en programas
cursos_en_programas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   program_course_online_id  253 non-null    int64 
 1   course_moodle_id          253 non-null    int64 
 2   conocimiento_id           253 non-null    int64 
 3   num_periodo               253 non-null    int64 
 4   program_id                253 non-null    int64 
 5   course_moodle_id.1        253 non-null    object
 6   conocimiento_name         253 non-null    object
 7   program_name              253 non-null    object
dtypes: int64(5), object(3)
memory usage: 15.9+ KB


In [16]:
fn.corregir_tipo(df= cursos_en_programas, columnas= ['program_id', 'program_category_id', ], tipo= 'category') #las pasamos a tipo category
fn.eliminar_unnamed(cursos_en_programas) #eliminamos columnas unnamed
cursos_en_programas.info() #correcto

#Se cambia el tipo de dato de las variables
for col in ['conocimiento_id','program_id','course_moodle_id.1']:
    cursos_en_programas[col] = cursos_en_programas[col].astype('category')#convertir las columnas seleccionadas en el for en categorical

KeyError: 'program_category_id'

In [88]:
#Itinerarios
itinearios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 467 entries, 0 to 466
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   active_flag         467 non-null    int64 
 1   category_id         467 non-null    int64 
 2   date_enroll         466 non-null    object
 3   enroll_end          170 non-null    object
 4   program_id          467 non-null    int64 
 5   remarks             3 non-null      object
 6   tutor_academico_id  467 non-null    int64 
 7   tutor_empresa_id    467 non-null    int64 
 8   user_id             467 non-null    int64 
 9   category_name       467 non-null    object
 10  program_name        467 non-null    object
dtypes: int64(6), object(5)
memory usage: 40.3+ KB


In [89]:
#En primer lugar, cambiar nombre a tabla.

itinerarios = itinearios

#Se cambia el tipo de dato de las variables
for col in ['category_id','program_id','tutor_academico_id','tutor_empresa_id', 'user_id']:
    itinerarios[col] = itinerarios[col].astype('category')#convertir las columnas seleccionadas en el for en categorical
    
#La variable date_enroll y  enroll_end  debemos pasarlo a una variable de tipo datetime
itinerarios['enroll_end'] = pd.to_datetime(itinerarios['enroll_end'], utc= True)
itinerarios['date_enroll'] = pd.to_datetime(itinerarios['date_enroll'], utc= True)

#Quitamos columnas Unnamed:
eliminar_unnamed(itinerarios)


In [90]:
#Notas_cursos
notas_cursos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3898 entries, 0 to 3897
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   enroll_grade_id  3898 non-null   int64  
 1   active_flag      3898 non-null   int64  
 2   course_id        3898 non-null   int64  
 3   enroll_id        3898 non-null   int64  
 4   grade            1475 non-null   float64
 5   online_progress  1518 non-null   float64
 6   course_name      3898 non-null   object 
 7   user_id          3898 non-null   int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 243.8+ KB


In [91]:
#Se cambia el tipo de dato de las variables
for col in ['enroll_grade_id','active_flag','course_id','enroll_id', 'user_id']:
    notas_cursos[col] = notas_cursos[col].astype('category')#convertir las columnas seleccionadas en el for en categorical

#Quitamos columnas Unnamed:
eliminar_unnamed(notas_cursos)

In [92]:
#Roles
roles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   role_id    4 non-null      int64 
 1   role_name  4 non-null      object
dtypes: int64(1), object(1)
memory usage: 192.0+ bytes


In [93]:
#Se cambia el tipo de dato de las variables
for col in ['role_id']:
    roles[col] = roles[col].astype('category')#convertir las columnas seleccionadas en el for en categorical

In [94]:
#Plantas
plantas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   planta_id    18 non-null     int64 
 1   planta_name  18 non-null     object
dtypes: int64(1), object(1)
memory usage: 416.0+ bytes


In [95]:
#Se cambia el tipo de dato de las variables
for col in ['planta_id']:
    plantas[col] = plantas[col].astype('category')#convertir las columnas seleccionadas en el for en categorical

In [96]:
#Accesos
accesos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8816 entries, 0 to 8815
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  8816 non-null   object
 1   timestamp    8816 non-null   object
 2   mdl_user_id  8816 non-null   int64 
 3   user_id      8816 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 275.6+ KB


In [97]:
#Se cambia el tipo de dato de las variables
for col in ['mdl_user_id','user_id']:
    accesos[col] = accesos[col].astype('category')#convertir las columnas seleccionadas en el for en categorical
    
##la variable timestamp debemos pasarlo a una variable de tipo datetime
accesos['timestamp'] = pd.to_datetime(accesos['timestamp'], utc= True)
accesos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8816 entries, 0 to 8815
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   description  8816 non-null   object             
 1   timestamp    8816 non-null   datetime64[ns, UTC]
 2   mdl_user_id  8816 non-null   category           
 3   user_id      8816 non-null   category           
dtypes: category(2), datetime64[ns, UTC](1), object(1)
memory usage: 195.4+ KB


In [98]:
#Acceso_1ero_ultimo
acceso_1ero_ultimo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372 entries, 0 to 371
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   mdl_user_id    372 non-null    int64 
 1   1er acceso     372 non-null    object
 2   ultimo acceso  372 non-null    object
dtypes: int64(1), object(2)
memory usage: 8.8+ KB


In [99]:
#Se cambia el tipo de dato de las variables
for col in ['mdl_user_id']:
    acceso_1ero_ultimo[col] = acceso_1ero_ultimo[col].astype('category')#convertir las columnas seleccionadas en el for en categorical

#quitamos columnas Unnamed:
eliminar_unnamed(acceso_1ero_ultimo)

In [100]:
#Actividades
actividades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3464 entries, 0 to 3463
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   mdl_user_id     3464 non-null   int64  
 1   course_name     3464 non-null   object 
 2   course_id       3464 non-null   int64  
 3   category        3464 non-null   object 
 4   actividad_name  3460 non-null   object 
 5   grade           2114 non-null   float64
 6   time            2129 non-null   object 
dtypes: float64(1), int64(2), object(4)
memory usage: 189.6+ KB


In [101]:
#Se cambia el tipo de dato de las variables
for col in ['mdl_user_id','course_id','category']:
    actividades[col] = actividades[col].astype('category')#convertir las columnas seleccionadas en el for en categorical
    
#la variable time debemos pasarlo a una variable de tipo datetime
actividades['time'] = pd.to_datetime(actividades['time'], utc= True)

#quitamos columnas Unnamed:
eliminar_unnamed(actividades)

### DUPLICADOS

In [102]:
#Quitamos filas duplicadas
eliminar_duplicados(usuarios)
eliminar_duplicados(tutores_perfil)
eliminar_duplicados(conocimientos)
eliminar_duplicados(cursos)
eliminar_duplicados(programas)
eliminar_duplicados(cursos_en_programas)
eliminar_duplicados(itinerarios)
eliminar_duplicados(notas_cursos)
eliminar_duplicados(roles)
eliminar_duplicados(accesos)
eliminar_duplicados(acceso_1ero_ultimo)
eliminar_duplicados(actividades)

#habia 3 df con filas duplicadas


In [103]:
#Identificamos las columnas duplicadas
columnas_duplicadas(usuarios)

TypeError: data type not understood

In [104]:
columnas_duplicadas(tutores_perfil)

NameError: name 'np' is not defined

In [105]:
columnas_duplicadas(conocimientos)

TypeError: data type not understood

In [106]:
columnas_duplicadas(cursos)

TypeError: data type not understood

In [107]:
columnas_duplicadas(programas)

TypeError: data type not understood

In [208]:
columnas_duplicadas(cursos_en_programas)

[]


In [ ]:
columnas_duplicadas(itinerarios)

In [209]:
columnas_duplicadas(notas_cursos)

[]


In [210]:
columnas_duplicadas(roles)

[]


In [211]:
columnas_duplicadas(accesos)

[]


In [214]:
columnas_duplicadas(acceso_1ero_ultimo)

[]


In [215]:
columnas_duplicadas(actividades)
#Ningun data frame tiene columnas duplicadas

[]


### MISSING VALUES

In [68]:
#Usuarios
eliminar_missings(usuarios)

NameError: name 'eliminar_missings' is not defined

In [ ]:
##Observamos que la columna last_acces está completamente vacia. Procedemos a quitarla.
usuarios.drop(columns = 'last_access', inplace = True)

##Asimismo, la columna de mdl_user_id, tiene 15 instancias con NaN. 

############################################################ojo que planteamos aqui!!!!!!!!!!!

In [ ]:
#Tutores_perfil
missing_data = tutores_perfil.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print('')

tutores_perfil = tutores_perfil.dropna() ##quitamos las filas que son NA
tutores_perfil ##vemos que las instancias con perfil_id = 0 se han eliminado

In [ ]:
#Conocimientos
missing_data = conocimientos.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print('')

#En esta tabla de conocimientos tenemos varias variables (color,description,refrencia y categoria_conocimiento_name)
#con missings. Vamos a tratarlas:

#La variable color no nos aporta ninguna informacion de valor. La podemos quitar
conocimientos.drop(columns = 'color', inplace = True)

#La variable description, describe de lo que trata el conocimiento. AL no contar la mayoria de datos, y , ademas, ncreemos que
#no nos aporta valor para futuros procesos, hemos decido tambien eliminarlo.
conocimientos.drop(columns = 'description', inplace = True)

#La variable referencia, es una variable muy parecida a la anterior. Por el mismo motivo, se decide eliminar dicha columna.
conocimientos.drop(columns = 'referencia', inplace = True)

#La variable categoria_conocimiento_name tiene 22 missings. Esto se debe a que la categoria_conocimiento_id es 0. 
#Es decir, no existe ni tiene datos. Por lo tanto, vamos a quitar las filas que son missings o lo q es lo mismo, Filas
# que tienen el valor de 0 en la variable categoria_conocimiento_id.

# conseguir nombre de los indices para el que la columna categoria conocimiento id tiene 
#valor 0(missings en categoira_conocimiento_name)
indexNames = conocimientos[ conocimientos['categoria_conocimiento_id'] == 0 ].index
# eliminar estas filas de indices del dataframe conocimientos
conocimientos.drop(indexNames , inplace=True)

conocimientos.info()#observamos que no tenemos missings

In [ ]:
#Cursos
missing_data = cursos.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print('')

#EN esta tabla solamente tenemos una variable con missings. La variable SUmmary.
#Esta variable no aporta informacion relevante para el proyecto , por lo que eliminamos la columna.
cursos.drop(columns = 'summary', inplace = True)

In [ ]:
#Programas
missing_data = programas.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print('')

In [ ]:
#En esta tabla, observamos que solamente existe una instancia con un missing en la variable program_category_id.
#Por lo tanto, quitamos unicamente esa fila puesto que no se puede imputar de ninguna manera ya que al tratarse de strings,
#la manera de imputar es coger los strings mas frecuentes, pero en este caso no tiene sentido realizar ese proceso.

programas.dropna(subset=["program_category_id"], inplace = True)

In [ ]:
##Asimismo, hemos observado anteriormente que cuenta con instancias en las que el nombre del programa tiene un nombre un tanto
##extraño por lo que se ha decidio quitar esas instancias que cuentan con nombres "inventados" y que no cuenta con datos reales.
##Es el caso de las intancias con nombres de programa tales como: "asdfasdf", "pp", "POPO", PIPI" etc.

#Ademas, destacar que la instancia eliminada anteriormente por tenen un missing en la categoria del programa, tiene como nombre
#del programa "pepe". Por lo que se ajusta a nuestra teoria de que no son datos reales y han sido rellenados para realizar la limpieza

#Para ello, se ha decidido observar la siguiente tabla, la tabla de cursos en programas, debido a que deben tener una fuerte
#relacion entre ambas. 

programas['program_name'].unique() #se observa el nombre de estos programas que resultan extraños e inventados.

In [ ]:
#Cursos en programas
missing_data = cursos_en_programas.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print('')


In [ ]:
#Como se puede observar, toda esta tabla esta completamente limpia. No obstante, nos sirve de gran ayuda para  poder solucionar
#los problemas de missings que disponiamos en la tabla de programas.

#Es decir, se van a juntar las dos tablas teniendo en cuenta los program_id de esta segunda tabla y observaremos que, mediante 
#esta tecnica eliminaremos aquellas instancias con un nombre de programa extraño tales como: "asdfasdf", "pp", "POPO", "PIPI" etc.
df_programas = cursos_en_programas.merge(programas, on="program_id", how = "left")#left join
display(df_programas.head())

In [ ]:
missing_data = df_programas.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print('')

In [ ]:
#Como se puede observar, ya no existen missings y las intancias "inventadas" con nombres de programas extraños estan eliminados.
#Sin embargo, queda por eliminar un par de instancias mas debido a que siguen existiendo nombre de programa llamados: "prueba_na",
#"prueba_na2", "prueba_na_v2", "prueba_01".

df_programas['program_name_x'].unique() #se observa el nombre de estos programas que resultan extraños e inventados.

#Obtenemos el nombre de estos y los insertamos en una lista: 'prueba_02', 'prueba_na',
      # 'prueba_na_v2', 'prueba_na2', 'pruebas_01', 'Sergio'

In [ ]:
nombres_programa_inventados = ['prueba_02', 'prueba_na','prueba_na_v2', 'prueba_na2', 'pruebas_01', 'Sergio']#Creacion lista 
                                                                                                            #nombres extraños
#eliminamos instancias que contienen elementos de la lista en la columna program_name
df_programas = df_programas[~df_programas.program_name_x.str.contains('|'.join(nombres_programa_inventados))]

In [ ]:
#Para terminar con estas dos tablas de programas, se procede a poner bien los nombres de las columnas y eliminar las que estan 
#duplicadas. Esto se debe al join que se ha realizado, solo interesa quedarse con una columna de program_name.
df_programas.drop(columns = 'program_name_y', inplace = True) #se elimina columna program_name_y

df_programas.rename(columns={'program_name_x': 'program_name', #se renombra la columna program_name_x en program_name
                            'course_moodle_id.1': 'course_moodle_id'},#se renombra columna course_moodle_id.1 encourse_moodle_id
                              inplace=True) 

In [ ]:
#Itinerarios
missing_data = itinerarios.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print('')

In [ ]:
#En esta tabla de itinerarios existen tres variables con missings. EN primer lugaar, la variable date_enroll, que cuenta solamente
#con un valor ausente.
itinerarios.iloc[57] #se selecciona esta instancia mencionada.
#Se ha observado que casualidad tiene 3 missings esta instancia por lo que la mejor decision es no contar con ella. 
#Se procede a eliminarla:
itinerarios.drop([57],axis=0, inplace = True)

#La segunda variable con missings, en este caso una variable con un numero de missings significante(297) es el enroll_end.
#Esta variable es el registro de la fecha fin del itinerario. Personalmente, creemos que estos valores no se pueden imputar de
#ninguna manera puesto que la fecha final de un itinerario es propio para cada uno y no se puede saber cual sera la fecha final.
#Asimismo, tampoco se cree conveniente tener que eliminar las 297 instancias que contienen missings puesto que el resto de la
#informacion puede resultar de gran utilidad.


# En cuanto a la tercera variable con missings, la variable remarks, mencionar que solamente consta de 3 instancias con alguna
#descripcion. Debido a ello, y a que se trata de una variable con poco valor para el trabajo a realizar, se procedera a eliminar
#la columna:
itinerarios.drop(columns = 'remarks', inplace = True) #se elimina columna remarks

In [ ]:
#Notas_cursos
missing_data = notas_cursos.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print('')

#Las dos variables con instancias que contienen missings son grade y online_progress. 

#En el caso de estas dos variables, es importante preguntarse porque tienen valores missings. La respuesta que se cree correcta
#es de que se tratan de variables que no tienen que tener un valor de obligatoriamente. Es decir, quizas algunos aprendices todavia 
# no han realizado el grado o el curso por lo que no tienen ninguna nota asociada. Lo mismo ocurre con el online_progress,
#puesto que mide el porcentaje de progreso del curso y quizas algunos aprendices no hayan dado comienzo al curso.

#Por estos motivos, se ha decidio que los valores ausentes que se encuentran en blanco se sustituyen por un 0 (sin iniciar.)
notas_cursos.fillna(0, inplace=True)#convertir NaN en 0

In [ ]:
#Roles
missing_data = roles.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print('')

#En esta tabla de roles no se cuenta con ningun valor ausente. Se trata de una tabla pequeña con informacion adicional 
#para el resto de tablas

In [ ]:
#Plantas
missing_data = plantas.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print('')

#En esta tabla de plantas no se cuenta con ningun valor ausente. Se trata de una tabla pequeña con informacion adicional 
#para el resto de tablas

In [ ]:
#Accesos
missing_data = accesos.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print('')

#En esta tabla, tambien vuelve a ocurrir que no se encuentra ningun valor ausente.

In [ ]:
#Acceso_1ero_ultimo
missing_data = acceso_1ero_ultimo.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print('')

## Aparentemente en esta tabla no hay NaN , pero se ha observado que algunas instancias tienen el siguiente valor "--". Por lo que
# se procedera a sustituir este valor y pasarlo a NaN para poder trabajar con estas instancias.

acceso_1ero_ultimo['1er acceso'].replace(['--'], 'NaN', inplace = True)#convertir '--' de segunda columna en NaN
acceso_1ero_ultimo['ultimo acceso'].replace(['--'], 'NaN', inplace = True)#convertir '--' de tecera columna en NaN

#Como estos valores no se pueden imputar, se van a coger los usuarios que han accedido al curso de la tabla accesos y se les añadria
#la informacion correspondiente de esta tabla en ellas.
#Es decir, los mdl_user_id de esta tabla que no tienen valores en sus accesos, tampoco estan en la tabla anterior de accesos.

df_accesos = accesos.merge(acceso_1ero_ultimo, on="mdl_user_id", how = "left")#left join
display(df_accesos.head())

In [ ]:
missing_data = df_accesos.isnull() #se comprueba el numero de missings despues del join
for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print('')

In [ ]:
##Todavia existen 18instancias con missings, esta respectivas instancias se procederan a eliminarlas puesto que no tenemos
#informacion que aporte valor para el trabajo.

df_accesos.dropna(subset=["1er acceso"], inplace = True) #eliminar na´s
df_accesos.dropna(subset=["ultimo acceso"], inplace = True)#eliminar na´s

In [ ]:
#Actividades
missing_data = actividades.isnull() #se comprueba el numero de missings despues del join
for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print('')

###En esta ultima tabla, existen tres variables con missings.

#La primera variable actividad_name,solamente tiene 4 instancias. Observando dichas, instancias nos hemos dado cuenta que 
#se tratan de 4 instancias que no cuentan con ninguna informacion. Es decir, en la variable grade y time tampoco tienen valor.
#Por lo que se eliminaran estas 4 instancias.
actividades.dropna(subset=["actividad_name"], inplace = True) #eliminar na´s

#La segunda variable es la de las notas . Estos valores no se pueden imputar debido a que se tratan de usuarios que no han comenzado
#con el curso , por lo que todavia no tienen ninguna nota.

#En cuanto a la ultima variable, la columna time consta de un numero elevado de missings (1331). En este caso, como es logico, 
#ocurre lo mismo. Aquellos usuarios que no hayan comenzado con la actividad, no tienen la fecha ni la nota correspondiente. 
#Por ello, no se cree oportuno eliminar todas las intancias y los mantendremos como NaN. (En caso de necesidad, posteriormente
#se eliminaran estos missings)

### TIPOS DE DATOS

In [219]:
#Usuarios
usuarios.dtypes

user_id               int64
useranonimo_name     object
active_flag           int64
language             object
last_access         float64
mdl_user_id         float64
role_admin            int64
role_manager          int64
role_student          int64
role_tutor            int64
planta_name          object
dtype: object

In [220]:
#Tutores_perfil
tutores_perfil.dtypes

perfil_id       int64
user_id         int64
perfil_name    object
dtype: object